In [2]:
import pandas as pd
import numpy as np


In [3]:
%%time

df = pd.read_parquet("preprocessed_books_2025_03_23.parquet")

df.head()

CPU times: user 1min 5s, sys: 10.2 s, total: 1min 15s
Wall time: 1min 7s


,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series,image_url_large,description_word_counts
0,0312853122,1,[],US,en,"[{'count': '3', 'name': 'to-read'}, {'count': ...",None,false,4.00,None,...,1984,https://www.goodreads.com/book/show/5333265-w-...,https://images.gr-assets.com/books/1310220028m...,5333265,3,5400751,W.C. Fields: A Life on Film,W.C. Fields: A Life on Film,https://images.gr-assets.com/books/1310220028l...,<NA>
1,0743509986,6,[],US,en,"[{'count': '2634', 'name': 'to-read'}, {'count...",None,false,3.23,B000FC0PBC,...,2001,https://www.goodreads.com/book/show/1333909.Go...,https://s.gr-assets.com/assets/nophoto/book/11...,1333909,10,1323437,Good Harbor,Good Harbor,https://s.gr-assets.com/assets/nophoto/book/11...,214
2,None,7,[189911],US,eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,false,4.03,None,...,1987,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140,8948723,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ...",https://images.gr-assets.com/books/1304100136l...,14
3,0743294297,3282,[],US,eng,"[{'count': '7615', 'name': 'to-read'}, {'count...",None,false,3.49,B002ENBLOK,...,2009,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184,6243154,Best Friends Forever,Best Friends Forever,https://s.gr-assets.com/assets/nophoto/book/11...,104
4,0850308712,5,[],US,en,"[{'count': '32', 'name': 'to-read'}, {'count':...",None,false,3.40,None,...,None,https://www.goodreads.com/book/show/287140.Run...,https://images.gr-assets.com/books/1413219371m...,287140,15,278577,Runic Astrology: Starcraft and Timekeeping in ...,Runic Astrology: Starcraft and Timekeeping in ...,https://images.gr-assets.com/books/1413219371l...,<NA>


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2074445 entries, 0 to 2074444
Data columns (total 31 columns):
 #   Column                   Dtype 
---  ------                   ----- 
 0   isbn                     object
 1   text_reviews_count       object
 2   series                   object
 3   country_code             object
 4   language_code            object
 5   popular_shelves          object
 6   asin                     object
 7   is_ebook                 object
 8   average_rating           object
 9   kindle_asin              object
 10  similar_books            object
 11  description              object
 12  format                   object
 13  link                     object
 14  authors                  object
 15  publisher                object
 16  num_pages                Int64 
 17  publication_day          object
 18  isbn13                   object
 19  publication_month        object
 20  edition_information      object
 21  publication_year         object

## Data cleaning

In [5]:
df[['description', 'description_word_counts']]

,description,description_word_counts
0,None,<NA>
1,"Anita Diamant's international bestseller ""The ...",214
2,Omnibus book club edition containing the Ladie...,14
3,Addie Downs and Valerie Adler were eight when ...,104
4,None,<NA>
...,...,...
2074440,Brigitte e feliz. Tem um trabalho de que gosta...,103
2074441,"The award-winning story of Britain, from the a...",153
2074442,Sir Arthur Conan Doyle is brought back to life...,75
2074443,"Gathers poems by William Blake, Emily Bronte, ...",24


### Handling missing values

In [6]:
art = pd.read_csv("data/ARTEMIS2.0/artemis_dataset_release_v0.csv")
art.describe()

,repetition
count,454684.000000
mean,8.533777
std,11.077652
min,5.000000
25%,5.000000
50%,5.000000
75%,6.000000
max,58.000000


In [7]:
len(art['emotion'].unique())

9

In [9]:
art['painting'].head().values

array(['vincent-van-gogh_portrait-of-madame-ginoux-l-arlesienne-1890',
       'vincent-van-gogh_portrait-of-madame-ginoux-l-arlesienne-1890',
       'vincent-van-gogh_portrait-of-madame-ginoux-l-arlesienne-1890',
       'vincent-van-gogh_portrait-of-madame-ginoux-l-arlesienne-1890',
       'vincent-van-gogh_portrait-of-madame-ginoux-l-arlesienne-1890'],
      dtype=object)

In [10]:
from sklearn.preprocessing import LabelEncoder

standard_path = "https://uploads1.wikiart.org/images/"
art['url'] = art['painting'].apply(lambda val: standard_path + val.replace('_', '/') + '.jpg')
art.head()


,art_style,painting,emotion,utterance,repetition,url
0,Post_Impressionism,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,something else,"She seems very happy in the picture, and you w...",10,https://uploads1.wikiart.org/images/vincent-va...
1,Post_Impressionism,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,sadness,This woman has really knotty hands which makes...,10,https://uploads1.wikiart.org/images/vincent-va...
2,Post_Impressionism,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,something else,"When looking at this woman, I am filled with c...",10,https://uploads1.wikiart.org/images/vincent-va...
3,Post_Impressionism,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,contentment,"A woman looking at ease, peaceful, and satisfi...",10,https://uploads1.wikiart.org/images/vincent-va...
4,Post_Impressionism,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,awe,She looks like a lady from that past that migh...,10,https://uploads1.wikiart.org/images/vincent-va...


In [9]:
import urllib.parse

def contains_special_chars(url):
    # Percent-encode the URL but leave standard characters safe.
    encoded = urllib.parse.quote(url, safe=':/')
    return url != encoded

# Assuming your DataFrame 'art' already has a 'url' column:
art['has_special_chars'] = art['url'].apply(contains_special_chars)
art_filtered = art[~art['has_special_chars']].copy()  # Keep rows without special characters

print("Original count:", len(art))
print("Filtered count:", len(art_filtered))


Original count: 454684
Filtered count: 434891


In [11]:
import urllib.parse
import pandas as pd
import torch
from torch.utils.data import Dataset
from PIL import Image, ImageFile
import requests
from io import BytesIO
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer, set_seed

seed = 949620
set_seed(seed)

# Convert the emotion column (strings) into integer labels in the range [0, 7]
label_encoder = LabelEncoder()
art_filtered['emotion_encoded'] = label_encoder.fit_transform(art_filtered['emotion'])
print("Emotion mapping:", dict(zip(label_encoder.classes_, range(len(label_encoder.classes_)))))
# Split the DataFrame into training and evaluation sets
train_df, eval_df = train_test_split(art_filtered, test_size=0.1, random_state=seed)

class ArtEmotionDataset(Dataset):
    def __init__(self, dataframe, feature_extractor):
        self.dataframe = dataframe.reset_index(drop=True)
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_url = row['url']
        # Percent-encode the URL to handle special characters
        encoded_url = urllib.parse.quote(image_url, safe=':/')
        try:
            response = requests.get(encoded_url)
            response.raise_for_status()  # Ensure we got a valid response
            image = Image.open(BytesIO(response.content)).convert("RGB")
        except Exception as e:
            # Option 1: Raise an error so you know which image failed
            print(f"Error loading image at {image_url}: {e}")
            # Option 2: Alternatively, log the error and return a default image/tensor
            image = Image.new("RGB", (224, 224), (0, 0, 0))  # a black dummy image

        # Process the image using the feature extractor
        encoding = self.feature_extractor(images=image, return_tensors="pt")
        # Remove the batch dimension
        encoding = {k: v.squeeze(0) for k, v in encoding.items()}
        # Use the encoded emotion label
        encoding["labels"] = torch.tensor(row['emotion_encoded'], dtype=torch.long)
        return encoding

# Load the feature extractor and initialize the model.
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=9)

# Create datasets for training and evaluation.
train_dataset = ArtEmotionDataset(train_df, feature_extractor)
eval_dataset = ArtEmotionDataset(eval_df, feature_extractor)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=1000,
    save_steps=1000,
    logging_steps=200,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=False,
    seed=seed
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

# Start training
trainer.train()

# Evaluate the model on the evaluation set
eval_results = trainer.evaluate()
print("Evaluation Results:")
print(eval_results)


Emotion mapping: {'amusement': 0, 'anger': 1, 'awe': 2, 'contentment': 3, 'disgust': 4, 'excitement': 5, 'fear': 6, 'sadness': 7, 'something else': 8}


/home/fharlaar/.local/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/fharlaar/.local/lib/python3.11/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/scratch-local/fharlaar.11049554/ipykernel_720085/2129290475.py:85: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_cl

Step,Training Loss,Validation Loss


Error loading image at https://uploads1.wikiart.org/images/oskar-kokoschka/not/detected/235871.jpg: 404 Client Error: Not Found for url: https://uploads1.wikiart.org/images/oskar-kokoschka/not/detected/235871.jpg
Error loading image at https://uploads1.wikiart.org/images/sir-lawrence-alma-tadema/pandora-1881.jpg: 404 Client Error: Not Found for url: https://uploads1.wikiart.org/images/sir-lawrence-alma-tadema/pandora-1881.jpg
Error loading image at https://uploads1.wikiart.org/images/paul-delaroche/the-children-of-edward-1831.jpg: 404 Client Error: Not Found for url: https://uploads1.wikiart.org/images/paul-delaroche/the-children-of-edward-1831.jpg
Error loading image at https://uploads1.wikiart.org/images/sir-lawrence-alma-tadema/prose-1879.jpg: 404 Client Error: Not Found for url: https://uploads1.wikiart.org/images/sir-lawrence-alma-tadema/prose-1879.jpg
Error loading image at https://uploads1.wikiart.org/images/salvador-dali/still-life.jpg: 404 Client Error: Not Found for url: http

KeyboardInterrupt: 